### Scrape corpus
https://archive.org/details/agathachristie/page/n5

In [1]:
#Scrape titles from html
from bs4 import BeautifulSoup
import requests
url='https://archive.org/details/agathachristie/'
soup_titles = BeautifulSoup(requests.get(url).content,'html.parser')

In [3]:
titles_w_path=[title['href'] for div in soup_titles.find_all('div',{'class':"jwrowV2"}) for title in div('a')]
len(titles_w_path)

154

In [4]:
titles_w_path[:] = [x for x in titles_w_path if not '/%D8%B9%D8%B1%D8%A8%D9%89/' in x]
len(titles_w_path)

60

In [5]:
titles_w_path[0]

'/stream/agathachristie/Agatha%20Christie/English/Agatha%20Christie%20-%204.50%20from%20Paddington'

In [17]:
relative_path_to_files='/stream/agathachristie/Agatha%20Christie/English/Agatha%20Christie%20-%20'
titles=[x.replace(relative_path_to_files,'').replace('%20','') for x in titles_w_path]
titles

['4.50fromPaddington',
 'ACaribbeanMystery',
 'APocketFullofRye',
 'AfterTheFuneral',
 'AppointmentWithDeath',
 'AtBertram%27sHotel',
 'BythePrickingofMyThumbs',
 'CardsontheTable',
 'CatAmongthePigeons',
 'DeathComesAsEnd',
 'DeathontheNile',
 'Dumbwitness',
 'ElephantsCanRemember',
 'English-MurderIsEasy',
 'EvilUndertheSun',
 'HalloweenParty',
 'HerculePoirot%27sCasebook',
 'HerculePoirot%27sChristmas',
 'HickoryDickoryDeath',
 'LordEdgwareDies',
 'MurderinMesopotamia',
 'MurderintheMews',
 'MurderontheLinks',
 'MurderontheOrientExpress',
 'MysteriousMrQuin',
 'Nemesis',
 'OneTwoBuckleMyShoe',
 'OrdealByInnocence',
 'ParkerPyneInvestigates',
 'PartnersinCrime',
 'PoirotLosesaClient',
 'Poirot%27sEarlyCases',
 'SadCypress',
 'SleepingMurder',
 'TheABCMurders',
 'TheAdventureofChristmasPudding',
 'TheBigFour',
 'TheCircularStaircase',
 'TheClocks',
 'TheGoldenBall',
 'TheHarlequinTeaSetAndOtherStories',
 'TheHollow.',
 'TheLaboursOfHercules',
 'TheListerdaleMysteryandOtherStories',
 '

In [18]:
exception='/stream/agathachristie/AgathaChristie/English/Christie_Agatha-'
titles=[x.replace(exception,'').replace('%27','') for x in titles]
titles[-2:]

['WhyDidntTheyAskEvans', 'MurderInThreeActs']

In [19]:
import os
try:
    os.mkdir('agatha_christie_corpus')
except FileExistsError:
    print("The directory",'agatha_christie_corpus',"already exists")
abs_url='https://archive.org'

The directory agatha_christie_corpus already exists


In [29]:
#Collect texts and store them as .txt
def finished(num_tries=0):
    return num_tries == 3
def store_texts(urlname,title):
    num_tries = 0
    txts=[]
    while not finished():
        soup_content = BeautifulSoup(requests.get(f'{abs_url}{titles_w_path[0]}_djvu.txt').content,'html.parser')
        try:
            with open(f'agatha_christie_corpus/{title}.txt', 'w',encoding='gb18030') as f:
                txt=soup_content.pre.get_text()
                #f.write(txt)
                txts.append(txt)
            return title
        except ValueError:
            num_tries += 1
            if not finished(num_tries):
                print("Event URL failed. Will retry...")
    raise RuntimeError("Error: Event URL failed.")

In [30]:
for (urlname,title) in zip(titles_w_path,titles):
     store_texts(urlname,title)

In [ ]:
# import re
# for n in files:
#     # Open each file
#     f = open(n,'r',encoding='utf-8-sig')
#     # Remove all non-alpha-numeric characters   
#     txt=re.sub(r'[0-9\n]+','',f.read())